In [22]:
import openai
from openai import OpenAI
import os
import base64
import requests
from key import OPENAI_API_KEY
import math
import numpy as np

In [23]:
# Initialize client
api_key=OPENAI_API_KEY
client = OpenAI(api_key=api_key)

In [24]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# Get url name
from video import extract_youtube_id
url = extract_youtube_id("https://www.youtube.com/watch?v=hFkJJkFIA2I")

# Get all the image paths
directory = './frames/' + url  # When you run video.py, all the frames located inside frames folder
image_paths = list_files(directory)

In [25]:
directory

'./frames/hFkJJkFIA2I'

In [26]:
# Get System Prompt
f = open("prompt.txt", "r")
prompt_text = f.read()

# Get User Input
user_input = "great moments"

In [27]:
# results = []

# # Iterate through each image path
# for image_path in image_paths:
#     # Getting the base64 string
#     base64_image = encode_image(image_path)

#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"
#     }

#     payload = {
#         "model": "gpt-4o",
#         "messages": [
#             {"role": "system", "content": prompt_text},
#             {"role": "user", "content": [{"type": "text", "text": user_input}, {"type": "image_url","image_url": {"url": f"data:image/jpg;base64,{base64_image}", "detail": "low"}}]}
#             # user_input text can be removed
#         ],
#         "max_tokens": 4096
#     }

#     # Make the API request
#     response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

#     # Append the result to the list
#     results.append(response.json())

In [28]:
# import concurrent.futures
# import base64
# import requests

# def make_api_request(image_path, api_key, prompt_text, user_input):
#     base64_image = encode_image(image_path)

#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"
#     }

#     payload = {
#         "model": "gpt-4o",
#         "messages": [
#             {"role": "system", "content": prompt_text},
#             {"role": "user", "content": [
#                 {"type": "text", "text": user_input},
#                 {"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{base64_image}", "detail": "low"}}
#             ]}
#         ],
#         "max_tokens": 4096
#     }

#     response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
#     return response.json()

# def process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=10):
#     results = []
#     with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#         future_to_image = {executor.submit(make_api_request, image_path, api_key, prompt_text, user_input): image_path for image_path in image_paths}
#         for future in concurrent.futures.as_completed(future_to_image):
#             image_path = future_to_image[future]
#             try:
#                 result = future.result()
#                 results.append(result)
#             except Exception as exc:
#                 print(f'{image_path} generated an exception: {exc}')

#     return results

# results = process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=10)
# print(results)

In [29]:
import concurrent.futures
import base64
import requests

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def make_api_request(image_paths, api_key, prompt_text, user_input):
    # Encode all images in the list
    base64_images = [encode_image(image_path) for image_path in image_paths]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Prepare the images for the payload
    images_payload = [{"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{img}", "detail": "low"}} for img in base64_images]

    payload = {
        "model": "gpt-4o",
        "messages": [
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": [
                {"type": "text", "text": user_input},
                *images_payload
            ]}
        ],
        "max_tokens": 1600
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=10, frames_per_request=3):
    results = []

    # Split image_paths into chunks of frames_per_request
    image_chunks = [image_paths[i:i + frames_per_request] for i in range(0, len(image_paths), frames_per_request)]

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_image_chunk = {executor.submit(make_api_request, chunk, api_key, prompt_text, user_input): chunk for chunk in image_chunks}
        for future in concurrent.futures.as_completed(future_to_image_chunk):
            image_chunk = future_to_image_chunk[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as exc:
                print(f'{image_chunk} generated an exception: {exc}')

    return results

results = process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=30, frames_per_request=3)
print(results)

[{'id': 'chatcmpl-9gynjqhHcYm1H9Y8PJI2CgBhpqYLT', 'object': 'chat.completion', 'created': 1720030523, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Explanation\n\n- **Main Subjects:** An older woman and a balcony inside a home.\n- **Actions:** The woman is peering over the balcony, waving with a smile, and appears to be engaged with someone or something below.\n- **Context:** The images capture a heartwarming moment within a domestic setting, suggesting a joyful interaction possibly with family or friends from a different level of the house, emphasizing the cozy and loving atmosphere.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1130, 'completion_tokens': 87, 'total_tokens': 1217}, 'system_fingerprint': 'fp_4008e3b719'}, {'id': 'chatcmpl-9gynkrtoY8mK6IOwfqBTZoSOhOwfh', 'object': 'chat.completion', 'created': 1720030524, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assi

In [30]:
# Result Sample
results[0]

{'id': 'chatcmpl-9gynjqhHcYm1H9Y8PJI2CgBhpqYLT',
 'object': 'chat.completion',
 'created': 1720030523,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '### Explanation\n\n- **Main Subjects:** An older woman and a balcony inside a home.\n- **Actions:** The woman is peering over the balcony, waving with a smile, and appears to be engaged with someone or something below.\n- **Context:** The images capture a heartwarming moment within a domestic setting, suggesting a joyful interaction possibly with family or friends from a different level of the house, emphasizing the cozy and loving atmosphere.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 1130,
  'completion_tokens': 87,
  'total_tokens': 1217},
 'system_fingerprint': 'fp_4008e3b719'}

In [31]:
# Iterate through all the results
missing_count = 0
missing_list = list()
for i in range(len(image_paths)//3):
    try:
        print(i, ": ", results[i]['choices'][0]['message']['content'])
    except KeyError:
        print(i, ": missing")
        missing_count += 1
        missing_list.append(i)
print("\nmissing count: ", missing_count)
print("missing list: ", missing_list)
# for i in range(len(image_paths)):
#     print(i, ": ", results[i]['choices'][0]['message']['content'])

0 :  ### Explanation

- **Main Subjects:** An older woman and a balcony inside a home.
- **Actions:** The woman is peering over the balcony, waving with a smile, and appears to be engaged with someone or something below.
- **Context:** The images capture a heartwarming moment within a domestic setting, suggesting a joyful interaction possibly with family or friends from a different level of the house, emphasizing the cozy and loving atmosphere.
1 :  ### Explanation

- **Main Subjects:** A person opening a door, a man inside a home, the interior of the house.
- **Actions:** The door is being unlocked and opened, revealing a man inside grinning broadly, followed by a partial view of the interior of the house.
- **Context:** The images capture a significant and joyful moment, potentially marking a surprise visit or reunion. The smile on the man's face highlights the happiness and excitement of the occasion, providing a warm and welcoming atmosphere.
2 :  #### Daily Vlog
- Main Subjects: T

In [32]:
embedding_model = 'text-embedding-3-small'
texts = results

def get_embedding(text, model=embedding_model):
    return client.embeddings.create(input=[text], model=model).data[0].embedding

def process_embeddings_in_parallel(results, model=embedding_model, num_threads=10):
    def embed_result(result):
        return get_embedding(result['choices'][0]['message']['content'], model=model)
    
    embedded_frame = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_index = {executor.submit(embed_result, result): i for i, result in enumerate(results)}
        for future in concurrent.futures.as_completed(future_to_index):
            i = future_to_index[future]
            try:
                embedding = future.result()
                embedded_frame[i] = np.array(embedding).reshape(1, -1)
            except Exception as exc:
                print(f'Embedding for result {i} generated an exception: {exc}')

    return embedded_frame

# Usage
embedded_frame = process_embeddings_in_parallel(texts, model=embedding_model, num_threads=30)

# Embed the query
embedded_query = get_embedding(user_input, model=embedding_model)
embedded_query = np.array(embedded_query).reshape(1, -1)

print(embedded_frame)
print(embedded_query)

{0: array([[ 0.05547855, -0.00577254, -0.00308234, ..., -0.01153801,
        -0.01749425, -0.00482576]]), 5: array([[ 0.00862612,  0.0336709 , -0.03754112, ..., -0.0237423 ,
        -0.02862473,  0.01528736]]), 3: array([[ 0.03340449,  0.02337427, -0.01707209, ..., -0.01133208,
        -0.03307902,  0.01701292]]), 10: array([[ 0.05892314,  0.02743414, -0.03640872, ..., -0.00618509,
        -0.01809094,  0.0059547 ]]), 12: array([[ 0.04150649,  0.00491879, -0.05339115, ..., -0.00614768,
        -0.02918545,  0.02234471]]), 1: array([[ 0.03587986,  0.02181461, -0.03786688, ...,  0.01064474,
        -0.02997558,  0.01195759]]), 2: array([[ 0.03232549,  0.01748163, -0.06263386, ..., -0.00192822,
        -0.04419541,  0.03113591]]), 16: array([[ 0.03936899,  0.01844265, -0.0417734 , ..., -0.007636  ,
        -0.02576159,  0.02187753]]), 4: array([[ 0.0551062 ,  0.01513878, -0.01090429, ..., -0.01085289,
        -0.02691054,  0.02574107]]), 6: array([[ 0.03431052,  0.01791988, -0.03172472, .

In [33]:
# embedding_model = 'text-embedding-3-small'
# def get_embedding(text, model=embedding_model):
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

# embedded_frame = {}
# for i in range(len(results)):
#     embedded_frame[i] = get_embedding(results[i]['choices'][0]['message']['content'], model=embedding_model)
#     embedded_frame[i] = np.array(embedded_frame[i]).reshape(1,-1)

# embedded_query = get_embedding(user_input, model=embedding_model)
# embedded_query = np.array(embedded_query).reshape(1,-1)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_result = {}
for i in range(len(image_paths)//3):
    try:
        similarity_result[i] = cosine_similarity(embedded_frame[i], embedded_query)
    except KeyError:
        similarity_result[i] = 0
        
ranked_frame_dict = dict(sorted(similarity_result.items(), key=lambda x:x[1], reverse=True))

In [35]:
ranked_frame_list = list(ranked_frame_dict.keys())
ranked_frame_list[:10]

[88, 143, 93, 67, 66, 73, 139, 1, 171, 107]

In [36]:
for i in ranked_frame_list[:10]:
    print(results[i]['choices'][0]['message']['content'])

#### Great Moments

- **Main Subjects:** A woman, a dog, and a man in a home setting.
- **Actions:** The woman is sitting on a couch, appearing relaxed and interacting with the dog. The dog moves across the couch energetically. The man is seen in the background initially standing and then busy with some task in another room.
- **Context:** The images capture a cozy, everyday moment in a home environment. The woman and her dog are enjoying a tranquil time on the couch, while the man is engaged in an activity nearby, adding to the domestic, relaxed atmosphere.
#### Great Moments

- **Main Subjects:** An elderly woman and her hands adorned with rings, another person observing and interacting.
- **Actions:** The elderly woman is showing off her collection of rings on her fingers. She adjusts the rings while another person closely observes and interacts, possibly asking questions or admiring the jewelry.
- **Context:** These images capture a tender and intimate moment where an elderly woman